[View in Colaboratory](https://colab.research.google.com/github/ugocupcic/LinuxMagSpacy/blob/master/LinuxMag.ipynb)

In [0]:
!pip install spacy
!python -m spacy download fr_core_news_md

# Extractions de différentes informations

Voici les tag de *part of speech* :


In [2]:
import spacy
nlp = spacy.load("fr_core_news_md")

doc = nlp(u"Bonjour le monde de Linux Mag")
print([(word.text, word.pos_) for word in doc])

[(u'Bonjour', u'NOUN'), (u'le', u'DET'), (u'monde', u'NOUN'), (u'de', u'ADP'), (u'Linux', u'PROPN'), (u'Mag', u'PROPN')]


Et maintenant, regardons les *Lemma*

In [3]:
Doc = nlp(u"Nous adorons ce magazine qui est très intéressant.")
print(" ".join([word.lemma_ + " " for  word in doc]))

Bonjour  le  monder  de  Linux  Mag 


Nous pouvons aussi identifier des groupes de mots qui représente un seul concept: les *noun chunks*.

In [4]:
doc = nlp(u"Nous allons parler d'intelligence artificielle.")

for noun_chunk in doc.noun_chunks:
    if " " in noun_chunk.text:
        print noun_chunk.text

d'intelligence artificielle


# Travail sur la similarité

## Distance entre des mots

Etudions la similarité de ces quelques mots:

In [5]:
fraise = nlp(u"fraise")
framboise = nlp(u"framboise")
journal = nlp(u"journal")

print "similarité fraise-framboise: " + str(fraise.similarity(framboise))
print "similarité fraise-journal: " + str(fraise.similarity(journal))
print "similarité framboise-journal: " + str(framboise.similarity(journal))

similarité fraise-framboise: 1.0000000315623996
similarité fraise-journal: -0.07691960231022113
similarité framboise-journal: -0.07691960231022113


## Distance entre des phrases

La mesure de similarité s'applique également à des phrases

In [6]:

phrase_1 = nlp(u"Ce journal est très intéressant.")
phrase_2 = nlp(u"Ce magazine est vraiment passionnant.")
phrase_3 = nlp(u"Je bois un thé en écrivant mon article.")

sim_1_2 = phrase_1.similarity(phrase_2)
sim_1_3 = phrase_1.similarity(phrase_3)
print "similarité entre les deux premières phrases: " + str(sim_1_2)
print "similarité entre la première et la dernière phrases: " + str(sim_1_3)


similarité entre les deux premières phrases: 0.8731641705836984
similarité entre la première et la dernière phrases: 0.6476781686527907


Mais elle présente des limitations:

In [7]:

phrase_1 = nlp(u"Ce journal est très intéressant.")
phrase_2 = nlp(u"Je trouve cette revue passionante.")
phrase_3 = nlp(u"Ce thé est vraiment délicieux.")

sim_1_2 = phrase_1.similarity(phrase_2)
sim_1_3 = phrase_1.similarity(phrase_3)
print "similarité entre les deux premières phrases: " + str(sim_1_2)
print "similarité entre la première et la dernière phrases: " + str(sim_1_3)


similarité entre les deux premières phrases: 0.6727251318633605
similarité entre la première et la dernière phrases: 0.8187913427933178


Simplifions nos phrases et recalculons des similarités:

In [8]:
def simplifier(sentence):
    sentence_as_NLP = nlp(sentence)
    simplified_sentence = ""
    for word in sentence_as_NLP:
        if word.pos_ in ["PROPN", "NOUN", "VERB", "ADJ"]:
            simplified_sentence += word.lemma_ + " "
            
    print ("Phrase complete: \"" + sentence + "\" => simplication: \"" + simplified_sentence + "\"")
    return simplified_sentence

def similarity(sentence1, sentence2):
    return nlp(sentence1).similarity(nlp(sentence2))

sentence1 = u"Ce thé est vraiment délicieux"
sentence2 = u"Je trouve cette revue passionante"
sentence3 = u"Ce thé est un délice."

print "----"
print "Similarité:" + str(similarity(simplifier(sentence1), simplifier(sentence2)))

print "----"
print "Similarité:" + str(similarity(simplifier(sentence1), simplifier(sentence3)))

----
Phrase complete: "Ce thé est vraiment délicieux" => simplication: "thé délicieux "
Phrase complete: "Je trouve cette revue passionante" => simplication: "trouver revu passionante "
Similarité:0.047787801947603605
----
Phrase complete: "Ce thé est vraiment délicieux" => simplication: "thé délicieux "
Phrase complete: "Ce thé est un délice." => simplication: "thé délice "
Similarité:1.0000000477387012


Nous avons toujours des limitations:

In [9]:

print "Similarité:" + str(similarity(simplifier(u"J'aime le thé."), simplifier(u"Je n'aime pas le thé")))


Phrase complete: "J'aime le thé." => simplication: "aimer thé "
Phrase complete: "Je n'aime pas le thé" => simplication: "aimer thé "
Similarité:1.0
